# PySpark SQL Queries 🔍

This notebook demonstrates how to use SQL queries in PySpark.

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark with SQL support
spark = SparkSession.builder \
    .appName('SQL Queries') \
    .master('spark://spark-master:7077') \
    .getOrCreate()

print('Spark session initialized with SQL support!')

## Create Sample Data

In [ ]:
# Customer data
customers = [
    (1, 'Alice', 'New York', 'Premium'),
    (2, 'Bob', 'Los Angeles', 'Standard'),
    (3, 'Charlie', 'Chicago', 'Premium'),
    (4, 'Diana', 'Houston', 'Standard'),
    (5, 'Eve', 'Phoenix', 'Premium')
]

# Orders data
orders = [
    (101, 1, 250.50, '2024-01-15'),
    (102, 2, 150.75, '2024-01-16'),
    (103, 1, 320.00, '2024-01-17'),
    (104, 3, 450.25, '2024-01-18'),
    (105, 2, 180.00, '2024-01-19'),
    (106, 4, 275.50, '2024-01-20'),
    (107, 1, 200.00, '2024-01-21'),
    (108, 5, 350.75, '2024-01-22')
]

# Create DataFrames
customers_df = spark.createDataFrame(
    customers, 
    ['customer_id', 'name', 'city', 'membership']
)

orders_df = spark.createDataFrame(
    orders, 
    ['order_id', 'customer_id', 'amount', 'order_date']
)

customers_df.show()
orders_df.show()

## Register Tables for SQL Queries

In [ ]:
# Register DataFrames as temporary views
customers_df.createOrReplaceTempView('customers')
orders_df.createOrReplaceTempView('orders')

print('Tables registered successfully!')

## Basic SQL Queries

In [ ]:
# SELECT all customers
spark.sql('SELECT * FROM customers').show()

# SELECT with WHERE clause
premium = spark.sql("""
    SELECT name, city 
    FROM customers 
    WHERE membership = 'Premium'
""")
premium.show()

## Aggregation Queries

In [ ]:
# Count orders per customer
order_counts = spark.sql("""
    SELECT customer_id, COUNT(*) as order_count
    FROM orders
    GROUP BY customer_id
    ORDER BY order_count DESC
""")
order_counts.show()

# Total revenue per customer
customer_revenue = spark.sql("""
    SELECT 
        customer_id,
        COUNT(*) as total_orders,
        SUM(amount) as total_spent,
        AVG(amount) as avg_order_value
    FROM orders
    GROUP BY customer_id
    ORDER BY total_spent DESC
""")
customer_revenue.show()

## JOIN Operations

In [ ]:
# INNER JOIN: Customer names with their orders
customer_orders = spark.sql("""
    SELECT 
        c.name,
        c.city,
        c.membership,
        o.order_id,
        o.amount,
        o.order_date
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
    ORDER BY o.order_date
""")
customer_orders.show()

# Aggregated JOIN
customer_summary = spark.sql("""
    SELECT 
        c.name,
        c.membership,
        COUNT(o.order_id) as total_orders,
        ROUND(SUM(o.amount), 2) as total_spent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name, c.membership
    ORDER BY total_spent DESC
""")
customer_summary.show()

## Subqueries

In [ ]:
# Customers with above-average spending
high_spenders = spark.sql("""
    SELECT 
        c.name,
        SUM(o.amount) as total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name
    HAVING SUM(o.amount) > (
        SELECT AVG(customer_total)
        FROM (
            SELECT SUM(amount) as customer_total
            FROM orders
            GROUP BY customer_id
        )
    )
""")
high_spenders.show()

## Window Functions

In [ ]:
# Rank orders by amount within each customer
ranked_orders = spark.sql("""
    SELECT 
        customer_id,
        order_id,
        amount,
        RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) as rank
    FROM orders
    ORDER BY customer_id, rank
""")
ranked_orders.show()

# Running total
running_total = spark.sql("""
    SELECT 
        order_date,
        amount,
        SUM(amount) OVER (ORDER BY order_date) as running_total
    FROM orders
    ORDER BY order_date
""")
running_total.show()

## Save Query Results

In [ ]:
# Save customer summary to parquet
# customer_summary.write.mode('overwrite').parquet('/home/jovyan/data/customer_summary.parquet')

print('SQL queries completed!')